# Mesh generation for tutorial 06, case 2b

This file generates the mesh which is used in the following examples:
* 2b_advection_diffusion_reaction

The test case is from section 5.2 of
```
F. Negri, G. Rozza, A. Manzoni and A. Quarteroni. Reduced Basis Method for Parametrized Elliptic Optimal Control Problems. SIAM Journal on Scientific Computing, 35(5): A2316-A2340, 2013.
```

In [ ]:
import pygmsh
from dolfinx import Mesh, MeshFunction, MeshValueCollection, MPI
from dolfinx.cpp.mesh import CellType, GhostMode
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Geometrical parameters and related quantities

In [ ]:
lcar = 0.05

### Create mesh

In [ ]:
class GenerateRectangleLines(object):
    def __init__(self, geom):
        self.geom = geom
        self.points = dict()
        self.lines = dict()

    def add_point(self, x, y):
        key = (x, y)
        try:
            return self.points[key]
        except KeyError:
            p = self.geom.add_point([x, y, 0.0], lcar)
            self.points[key] = p
            return p

    def add_line(self, p0, p1):
        try:
            return self.lines[(p0, p1)]
        except KeyError:
            l01 = self.geom.add_line(p0, p1)
            self.lines[(p0, p1)] = l01
            self.lines[(p1, p0)] = -l01
            return l01

    def __call__(self, x_min, x_max, y_min, y_max):
        p0 = self.add_point(x_min, y_min)
        p1 = self.add_point(x_max, y_min)
        if isinstance(y_max, list):
            p2 = [self.add_point(x_max, y) for y in y_max]
            p3 = self.add_point(x_min, y_max[-1])
        else:
            p2 = self.add_point(x_max, y_max)
            p3 = self.add_point(x_min, y_max)
        l0 = self.add_line(p0, p1)
        if isinstance(y_max, list):
            p1_p2 = [p1] + p2
            l1 = [self.add_line(p1_p2[i], p1_p2[i + 1]) for i in range(len(p2))]
            l2 = self.add_line(p2[-1], p3)
        else:
            l1 = self.add_line(p1, p2)
            l2 = self.add_line(p2, p3)
        l3 = self.add_line(p3, p0)
        return (l0, l1, l2, l3)


geom = pygmsh.built_in.Geometry()
generate_rectangle_lines = GenerateRectangleLines(geom)
[l0, l1, l2, l3] = generate_rectangle_lines(0.0, 1.0, 0.0, 1.0)
[l4, l5, l6, _] = generate_rectangle_lines(1.0, 2.5, 0.0, [0.3, 0.7, 1.0])
[l7, l8, l9, l10] = generate_rectangle_lines(0.2, 0.8, 0.3, 0.7)
[l11, l12, l13, l14] = generate_rectangle_lines(1.2, 2.5, 0.3, 0.7)
geom.add_physical([l0, l2, l3], label=1)
geom.add_physical([l4, l6], label=2)
geom.add_physical([l5[0], l5[1], l5[2]], label=3)
line_loop_rectangle_outer_left = geom.add_line_loop([l0, l1, l2, l3])
line_loop_rectangle_outer_right = geom.add_line_loop([l4, l5[0], l5[1], l5[2], l6, -l1])
line_loop_rectangle_inner_left = geom.add_line_loop([l7, l8, l9, l10])
line_loop_rectangle_inner_right = geom.add_line_loop([l11, l12, l13, l14])
rectangle_outer_left = geom.add_plane_surface(line_loop_rectangle_outer_left, [line_loop_rectangle_inner_left])
rectangle_outer_right = geom.add_plane_surface(line_loop_rectangle_outer_right, [line_loop_rectangle_inner_right])
rectangle_inner_left = geom.add_plane_surface(line_loop_rectangle_inner_left)
rectangle_inner_right = geom.add_plane_surface(line_loop_rectangle_inner_right)
geom.add_physical(rectangle_outer_left, label=13)
geom.add_physical(rectangle_outer_right, label=14)
geom.add_physical(rectangle_inner_left, label=11)
geom.add_physical(rectangle_inner_right, label=12)
pygmsh_mesh = pygmsh.generate_mesh(geom)
mesh = Mesh(MPI.comm_world, CellType.triangle, pygmsh_mesh.points[:, :2],
            pygmsh_mesh.cells_dict["triangle"], [], GhostMode.none)

In [ ]:
plot(mesh)

### Create subdomains

In [ ]:
subdomains_mvc = MeshValueCollection("int", mesh, mesh.topology.dim, pygmsh_mesh.cells_dict["triangle"],
                                     pygmsh_mesh.cell_data_dict["gmsh:physical"]["triangle"] - 10)
subdomains = MeshFunction("int", mesh, subdomains_mvc, 0)

### Create boundaries

In [ ]:
boundaries_mvc = MeshValueCollection("int", mesh, mesh.topology.dim - 1, pygmsh_mesh.cells_dict["line"],
                                     pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"])
boundaries = MeshFunction("int", mesh, boundaries_mvc, 0)

### Save

In [ ]:
with XDMFFile(MPI.comm_world, "graetz_1.xdmf") as output:
    output.write(mesh)
with XDMFFile(MPI.comm_world, "graetz_1_subdomains.xdmf") as output:
    output.write(subdomains)
with XDMFFile(MPI.comm_world, "graetz_1_boundaries.xdmf") as output:
    output.write(boundaries)